<a href="https://colab.research.google.com/github/WebberMark02/machine-learning-project/blob/main/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importo le librerie necessarie e scelgo di utilizzare "tensorflow" come
backend per "Keras".

In [1]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import keras
from matplotlib import pyplot as plt
import sklearn
from tensorflow.keras.datasets import mnist, fashion_mnist

Imposto le variabili globali.

In [2]:
seed = 42
training_set_size = 250000
testing_set_size = 5000
validation_set_size = 20000
image_shape = (32, 32)
batch_size = 256
epochs = 250
learning_rate = 0.001
early_stopping_patience = 50

Definisco il generatore di immagini sulle quali il modello
verrà addestrato e testato.  
Il generatore restituisce "batchsize" immagini; ogni immagine è
la media di due immagini scelte casualmente rispettivamente da 'x1' e 'x2'.  
Il generatore restituisce, inoltre, per ogni media di immagini, la coppia delle immagini delle quali è stata calcolata la media stessa.

In [3]:
def datagenerator(x1,x2,batchsize):
    n1 = x1.shape[0]
    n2 = x2.shape[0]
    while True:
        num1 = np.random.randint(0, n1, batchsize)
        num2 = np.random.randint(0, n2, batchsize)

        x_data = (x1[num1] + x2[num2]) / 2.0
        y_data = (x1[num1], x2[num2])

        yield x_data, y_data

Definisco una funzione per il controllo del bilanciamento delle classi.  
Mi servirà per verificare che la divisione stratificata abbia avuto successo.

In [4]:
def stampa_percentuale( y, title="" ):
  if title:print(title)
  # Calcoliamo le occorrenze di ciascuna classe nel dataset
  unique, counts = np.unique(y, return_counts=True)
  percentuali = (counts / len(y)) * 100 # calcolo la % di occorrenze per ciascuna classe
  # Stampiamo le occorrenze e le percentuali
  for classe, conteggio, percentuale in zip(unique, counts, percentuali):
      print(f"Classe {classe}: Occorrenze = {conteggio}, Percentuale {percentuale} %" )
  print(f"Totale occorrenze : {sum(counts)}")
  print()

Definisco un modello banale.
Mi servirà per valutare che la rete abbia prestazioni migliori di esso.

In [5]:
def ide_model(x):
   return((x,x))

Ora ha inizio la fase di caricamento e preparazione dei dataset che verranno utilizzati
per addestrare e esaminare le prestazioni della rete.

Prima di tutto, carico i training set e i testing set di "MNIST" e "Fashion MNIST".

In [6]:
(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()
(fashion_mnist_x_train, fashion_mnist_y_train), (fashion_mnist_x_test, fashion_mnist_y_test) = fashion_mnist.load_data()

print(np.shape(mnist_x_train))

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
(60000, 28, 28)


Ridimensiono le immagini tramite padding, portando la loro risoluzione da 28x28 a 32x32.  
Inoltre, le normalizzo nell'intervallo [0, 1] così da poterle dare in input alla rete neurale che costruirò.

In [7]:
#normalize in and pad
mnist_x_train = np.pad(mnist_x_train,((0,0),(2,2),(2,2)))/255.
mnist_x_test = np.pad(mnist_x_test,((0,0),(2,2),(2,2)))/255.
fashion_mnist_x_train = np.pad(fashion_mnist_x_train,((0,0),(2,2),(2,2)))/255.
fashion_mnist_x_test = np.pad(fashion_mnist_x_test,((0,0),(2,2),(2,2)))/255.

print(np.shape(mnist_x_train))

(60000, 32, 32)


Controllo il bilanciamento delle classi nei training set prima della divisione stratificata.

In [8]:
stampa_percentuale(mnist_y_train, 'MNIST training set completo')
stampa_percentuale(fashion_mnist_y_train, 'Fashion MNIST training set completo')

MNIST training set completo
Classe 0: Occorrenze = 5923, Percentuale 9.871666666666666 %
Classe 1: Occorrenze = 6742, Percentuale 11.236666666666666 %
Classe 2: Occorrenze = 5958, Percentuale 9.93 %
Classe 3: Occorrenze = 6131, Percentuale 10.218333333333334 %
Classe 4: Occorrenze = 5842, Percentuale 9.736666666666666 %
Classe 5: Occorrenze = 5421, Percentuale 9.035 %
Classe 6: Occorrenze = 5918, Percentuale 9.863333333333333 %
Classe 7: Occorrenze = 6265, Percentuale 10.441666666666666 %
Classe 8: Occorrenze = 5851, Percentuale 9.751666666666667 %
Classe 9: Occorrenze = 5949, Percentuale 9.915000000000001 %
Totale occorrenze : 60000

Fashion MNIST training set completo
Classe 0: Occorrenze = 6000, Percentuale 10.0 %
Classe 1: Occorrenze = 6000, Percentuale 10.0 %
Classe 2: Occorrenze = 6000, Percentuale 10.0 %
Classe 3: Occorrenze = 6000, Percentuale 10.0 %
Classe 4: Occorrenze = 6000, Percentuale 10.0 %
Classe 5: Occorrenze = 6000, Percentuale 10.0 %
Classe 6: Occorrenze = 6000, Perc

Divido ogni training set in due insiemi: il training set e il validation set.
Le immagini dei validation set verranno usate per l'ottimizzazione degli iper-parametri della rete.
Ogni validation set conterrà il 20% delle immagini del training set di partenza.
Uso la stratificazione per mantenere le classi nelle stesse proporzioni.

In [9]:
mnist_x_train, mnist_x_val, mnist_y_train, mnist_y_val = sklearn.model_selection.train_test_split(mnist_x_train, mnist_y_train, test_size=0.2, stratify=mnist_y_train, random_state=seed)
fashion_mnist_x_train, fashion_mnist_x_val, fashion_mnist_y_train, fashion_mnist_y_val = sklearn.model_selection.train_test_split(fashion_mnist_x_train, fashion_mnist_y_train, test_size=0.2, stratify=fashion_mnist_y_train, random_state=seed)

Controllo il bilanciamento delle classi nei training set e nei validation set ottenuti dalla divisione stratificata.

In [10]:
stampa_percentuale(mnist_y_train, 'MNIST training set risultante')
stampa_percentuale(mnist_y_val, 'MNIST validation set risultante')

stampa_percentuale(fashion_mnist_y_train, 'Fashion MNIST training set risultante')
stampa_percentuale(fashion_mnist_y_val, 'Fashion MNIST validation set risultante')

MNIST training set risultante
Classe 0: Occorrenze = 4738, Percentuale 9.870833333333334 %
Classe 1: Occorrenze = 5394, Percentuale 11.2375 %
Classe 2: Occorrenze = 4766, Percentuale 9.929166666666667 %
Classe 3: Occorrenze = 4905, Percentuale 10.21875 %
Classe 4: Occorrenze = 4674, Percentuale 9.7375 %
Classe 5: Occorrenze = 4337, Percentuale 9.035416666666666 %
Classe 6: Occorrenze = 4734, Percentuale 9.8625 %
Classe 7: Occorrenze = 5012, Percentuale 10.441666666666666 %
Classe 8: Occorrenze = 4681, Percentuale 9.752083333333333 %
Classe 9: Occorrenze = 4759, Percentuale 9.914583333333333 %
Totale occorrenze : 48000

MNIST validation set risultante
Classe 0: Occorrenze = 1185, Percentuale 9.875 %
Classe 1: Occorrenze = 1348, Percentuale 11.233333333333333 %
Classe 2: Occorrenze = 1192, Percentuale 9.933333333333334 %
Classe 3: Occorrenze = 1226, Percentuale 10.216666666666667 %
Classe 4: Occorrenze = 1168, Percentuale 9.733333333333333 %
Classe 5: Occorrenze = 1084, Percentuale 9.033